In [79]:
import pandas as pd
import numpy as np
df=pd.read_csv("combined.csv")

def convertToFloat(x):
    x=x.strip()
    if x=="-" or x=="NR" or x=='' or x=="." or x=="" or x==' ':
        return 0.01
    else:
        
        return float(x)

In [94]:
df["rain"]=df["Rain"].apply(convertToFloat)
import datetime
df["month"]=df["Date"].apply(lambda x:datetime.datetime.strptime(x,"%Y-%m-%d").month)
df["year"]=df["Date"].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").year)
df["day"]=df["Date"].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").day)

In [95]:
def isRain(x):
    if x>100:
        return 5
    if x>50:
        return 4
    if x>25:
        return 3
    if x>2:
        return 2
    elif x!=0.01:
        return 1
 
    else:
        return 0

In [96]:
df["is_rain"]=df["rain"].apply(isRain)

In [97]:
dummydf=pd.get_dummies(df[["District","rain","month","day","year","is_rain"]])

In [48]:
Al_df=df[df.District=="Alangayam"][["rain","month","day","year","is_rain"]]

In [98]:
y=dummydf.pop("rain")
x=dummydf

In [148]:
y=Al_df.pop("rain")
x=Al_df
X_train, X_test, y_train, y_test = train_test_split(x , y, test_size=0.33, random_state=42) 

In [99]:
 from sklearn.model_selection import train_test_split
 from sklearn.preprocessing import MinMaxScaler

In [100]:
X_train, X_test, y_train, y_test = train_test_split(x , y, test_size=0.33, random_state=42)

In [38]:
scaler = MinMaxScaler()
tempdf=pd.DataFrame(scaler.fit_transform(X_train[["month","day","year"]].values))
X_train["month"]=tempdf[0]
X_train["day"]=tempdf[1]
X_train["year"]=tempdf[2]

scalertest=MinMaxScaler()
tempdf=pd.DataFrame(scaler.fit_transform(X_test[["month","day","year"]].values))
X_test["month"]=tempdf[0]
X_test["day"]=tempdf[1]
X_test["year"]=tempdf[2]

# X_train=scaler.fit_transform(X_train)
# scaler2 = StandardScaler()

# X_test=scaler2.fit_transform(X_test)

<ipython-input-38-227f16517e95>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["month"]=tempdf[0]
<ipython-input-38-227f16517e95>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train["day"]=tempdf[1]
<ipython-input-38-227f16517e95>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [19]:
tempdf[1]

0        0.366667
1        0.033333
2        0.666667
3        0.066667
4        0.600000
           ...   
31010    0.666667
31011    0.166667
31012    0.166667
31013    0.633333
31014    0.000000
Name: 1, Length: 31015, dtype: float64

In [21]:
import xgboost as xg

In [ ]:
!pip3 install xgboost

In [107]:
from sklearn.linear_model import LinearRegression

In [108]:
reg = LinearRegression().fit(X_train, y_train)

In [109]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test,reg.predict(X_test))

2.2034639652025696

-1.404206625853364

In [104]:
reg.predict(X_train)

array([ 0.21894508, 12.46740162, -0.14647121, ..., 12.94425225,
        0.38613749,  0.3628996 ])

In [31]:
import xgboost as xg  

In [118]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.]])

In [101]:
xgb_r = xg.XGBRegressor(objective ='reg:linear',    
                  n_estimators = 50, seed = 123,max_depth=20,eta=0.7)
   
# Fitting the model
xgb_r.fit(X_train, y_train)

[00:05:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, eta=0.7, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.699999988, max_delta_step=0, max_depth=20,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=50, n_jobs=12, num_parallel_tree=1,
             objective='reg:linear', random_state=123, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=123, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [102]:
pred = xgb_r.predict(X_test)


C:\Users\om purohit\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [35]:
def mean_absolute_percentage_error(y_true, y_pred): 
    # y_true, y_pred = check_arrays(y_true, y_pred)

    ## Note: does not handle mix 1d representation
    #if _is_1d(y_true): 
    #    y_true, y_pred = _check_1d_array(y_true, y_pred)

    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [41]:
mean_absolute_percentage_error( y_train,pred)

inf

In [105]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_test,pred)

0.8999667648096353

In [173]:
pred_test=xgb_r.predict(X_test)

C:\Users\om purohit\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


In [43]:
mean_absolute_error(y_test,pred_test)

NameError: name 'pred_test' is not defined

In [103]:
d={"orig":y_test,"pred_test":pred}
preddf=pd.DataFrame(d)

In [104]:
preddf

,orig,pred_test
35844,0.01,0.010001
18346,0.01,0.010001
8140,56.00,60.217617
43290,0.01,0.010001
40376,0.01,0.010001
...,...,...
1274,0.01,0.010001
13166,0.01,0.010001
45618,0.01,0.010001
44814,0.01,0.010001


In [179]:
from sklearn.svm import LinearSVR
eps = 5
svr = LinearSVR(epsilon=eps, C=0.01, fit_intercept=True)

In [180]:
svr.fit(X_train,y_train)

C:\Users\om purohit\anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVR(C=0.01, epsilon=5)

In [183]:
mean_absolute_error(y_test,svr.predict(X_test))

5.816683235951595

In [186]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test,pred)

50.42033054947053

In [106]:
mean_absolute_error(y_train,xgb_r.predict(X_train))

C:\Users\om purohit\anaconda3\lib\site-packages\xgboost\data.py:112: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  warnings.warn(


0.0006024791217659378